In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import random
from collections import defaultdict
import json

import wandb

from sklearn.cluster import KMeans

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
no_bubble_label = np.load("/home/cygnus/fujimoto/Cygnus-X_CAE/data/zroing_resize_data/resize_data/condition_match/bubble_region_label.npy")

In [3]:
np.sum(no_bubble_label == 0)

5380

In [4]:
api = wandb.Api()
artifact_path = 'yudai22-/condition_matching/training_log:v3'
artifact = api.artifact(artifact_path)

run = artifact.logged_by()


try:
    config_dict = json.loads(run.config)
    latent = config_dict["latent_num"]
    print(latent)
except json.JSONDecodeError:
    # run.config が有効なJSON文字列でなかった場合の処理
    print("エラー: run.config は有効なJSON形式ではありません。")
except TypeError:
    # run.config が文字列でなかった場合の処理 (念のため)
    print("エラー: run.config は文字列型ではありません。")


# latent = run.config["latent_num"]
print(latent)


match = re.findall('/.*/', artifact_path)
log_name = re.search(r"(training.*)", artifact_path).group()

print(log_name)
print(match[0][1:-1])

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: Paste an API key from your profile and hit enter:

  ········


wandb: Currently logged in as: yudai22 (yudai22-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'value': 100}
{'value': 100}
training_log:v3
condition_matching


In [5]:
all_region_latent = np.load(f"/home/cygnus/fujimoto/Cygnus-X_CAE/analysis/check_latent/latent_list/{match[0][1:-1]}/all/{log_name}_all_latent.npy")
bubble_region_latent = np.load(f"/home/cygnus/fujimoto/Cygnus-X_CAE/analysis/check_latent/latent_list/{match[0][1:-1]}/bubble/{log_name}_bubble_latent.npy")
view_all_region = np.load("/home/cygnus/fujimoto/Cygnus-X_CAE/data/zroing_resize_data/resize_data/condition_match/vflip/CygnusX_cut_truncation_3size_resize_to_mode_percentile_100x100.npy")
bubble_region = np.load("/home/cygnus/fujimoto/Cygnus-X_CAE/data/zroing_resize_data/resize_data/condition_match/vflip/CygnusX_bubble_vflip.npy")

In [6]:
n_clusters = 3
km = KMeans(n_clusters=n_clusters, init='k-means++',random_state=0)
points = np.r_[all_region_latent, bubble_region_latent]
kmh = km.fit_predict(points)

In [7]:
no_bubble_label_list = list(no_bubble_label)
no_bubble_label_list += [0]*len(bubble_region)

In [8]:
len(no_bubble_label_list)

7956

In [9]:
khm_remove = []
for cluster, label in zip(kmh, no_bubble_label_list):
    if label == 1:
        cluster = np.nan
    else:
        pass
    khm_remove.append(cluster)

In [10]:
nan_num = np.sum(np.isnan(khm_remove))
print(len(khm_remove))
print(nan_num)

7956
2545


In [11]:
c_bubble = khm_remove[-31:]
c_all = khm_remove[:-31]
print(len(c_bubble))
print(len(c_all))

31
7925


In [12]:
all_num = len(c_all) - nan_num

for unique_num in np.unique(khm_remove):
    if np.isnan(unique_num):
        continue
    val_all = np.sum(c_all == unique_num)
    val_bubble = np.sum(c_bubble == unique_num)
    
    print(f"| cluster{int(unique_num)} | {'{:.1f}'.format(val_all/all_num*100)}% | {'{:.1f}'.format(val_bubble/len(c_bubble)*100)}% | {val_all}個 | {val_bubble}個 |")

| cluster0 | 22.5% | 32.3% | 1208個 | 10個 |
| cluster1 | 56.8% | 22.6% | 3056個 | 7個 |
| cluster2 | 20.7% | 45.2% | 1116個 | 14個 |


# 全データ(Bubble除く)のクラスタリング結果出力

In [19]:
all_labeled_indices = defaultdict(list)

for i, label in enumerate(c_all):
    if np.isnan(label):
        continue

    all_labeled_indices[label].append(i)
# labeled_indices = sorted(labeled_indices.items())

# 結果を表示
for label, indices in all_labeled_indices.items():
    print(f"ラベル{label}のインデックス ({len(indices)}個)")

ラベル1のインデックス (408個)
ラベル3のインデックス (2230個)
ラベル4のインデックス (1548個)
ラベル2のインデックス (785個)
ラベル0のインデックス (409個)


In [20]:
output_num = 10

random_per_cluster = []
for cluster_num in range(len(all_labeled_indices)):
    selected_indices = random.sample(all_labeled_indices[cluster_num], output_num)
    selected_indices = sorted(selected_indices)
    random_per_cluster.append(selected_indices)
    
print(random_per_cluster)

[[1968, 2287, 2298, 3150, 3158, 3505, 4371, 4796, 5573, 5828], [579, 1648, 1938, 2046, 3391, 4121, 4390, 4502, 6881, 7063], [540, 2110, 3603, 3799, 4059, 4580, 5589, 6027, 6140, 7123], [303, 906, 1770, 4654, 5757, 6261, 6370, 7061, 7092, 7676], [783, 1874, 1928, 2304, 2463, 2519, 4935, 4995, 5543, 7603]]


In [21]:
for cluster_num in range(len(all_labeled_indices)):
    selected_indices = random_per_cluster[cluster_num]
    
    rows = output_num // 10
    fig, axes = plt.subplots(rows, 10, figsize=(2*10, 3*rows))
    fig.suptitle(f"Cluster {cluster_num}", fontsize=20, color="r")

    for k, ax in enumerate(axes.flatten()):
        img_idx = selected_indices[k]
        ax.imshow(np.sum(view_all_region[img_idx], axis=0), cmap="jet", vmin=0, vmax=5)
        ax.axis("off")
        ax.set_title(f"No:{img_idx}")

    # レイアウトを調整して、タイトルと画像が重ならないようにする
    plt.tight_layout()
    plt.savefig(f"{match[0][1:-1]}_{log_name}_integ_cluster{cluster_num}.png")
    plt.show()

In [22]:
for cluster_num in range(len(all_labeled_indices)):
    selected_indices = random_per_cluster[cluster_num]
    
    rows = len(selected_indices)
    per_colums = len(view_all_region[0])
    
    fig, axes = plt.subplots(rows, per_colums, figsize=(3*per_colums, 4*rows))
    fig.suptitle(f"Cluster {cluster_num}", fontsize=50, color="r")

    for i, index in enumerate(selected_indices):
        data = view_all_region[index]
        
        for j, sliced_data in enumerate(data):
            if j == 0:
                axes[i, j].set_title(f"No:{index}", fontsize=30)
            
            axes[i, j].imshow(sliced_data, cmap="jet", vmin=0, vmax=1)
            axes[i, j].axis("off")
            
    # レイアウトを調整して、タイトルと画像が重ならないようにする
    plt.tight_layout(rect=[0, 0.03, 1, 0.98])
    plt.savefig(f"{match[0][1:-1]}_{log_name}_cluster{cluster_num}.png", dpi=70)
    plt.show()

# Bubbleのクラスタリング結果

In [23]:
bubble_labeled_indices = defaultdict(list)

for i, label in enumerate(c_bubble):
    bubble_labeled_indices[label].append(i)

# 結果を表示
for label, indices in bubble_labeled_indices.items():
    print(f"ラベル{label}のインデックス ({len(indices)}個)")

ラベル2のインデックス (9個)
ラベル1のインデックス (7個)
ラベル3のインデックス (6個)
ラベル0のインデックス (4個)
ラベル4のインデックス (5個)


In [24]:
bubble_labeled_indices

defaultdict(list,
            {2: [0, 5, 6, 9, 10, 12, 16, 26, 29],
             1: [1, 3, 4, 7, 14, 20, 28],
             3: [2, 11, 13, 17, 18, 21],
             0: [8, 19, 22, 27],
             4: [15, 23, 24, 25, 30]})

In [25]:
bubble_per_cluster = []
for cluster_num in range(len(bubble_labeled_indices)):
    selected_indices = bubble_labeled_indices[cluster_num]
    selected_indices = sorted(selected_indices)
    bubble_per_cluster.append(selected_indices)

In [26]:
for cluster_num in range(len(bubble_per_cluster)):
    selected_indices = bubble_per_cluster[cluster_num]
    
    rows = (len(selected_indices) + 9) // 10
    fig, axes = plt.subplots(rows, 10, figsize=(2*10, 3*rows))
    fig.suptitle(f"Cluster {cluster_num}", fontsize=20, color="r")

    for k, ax in enumerate(axes.flatten()):
        if k < len(selected_indices):
            img_idx = selected_indices[k]
            ax.imshow(np.sum(bubble_region[img_idx], axis=0), cmap="jet", vmin=0, vmax=5)
            ax.set_title(f"No:{img_idx}")
        ax.axis("off")

    # レイアウトを調整して、タイトルと画像が重ならないようにする
    plt.tight_layout()
    plt.savefig(f"{match[0][1:-1]}_{log_name}_bubble_integ_cluster{cluster_num}.png")
    plt.show()

In [27]:
for cluster_num in range(len(bubble_per_cluster)):
    selected_indices = bubble_per_cluster[cluster_num]
    
    rows = len(selected_indices)
    per_colums = len(view_all_region[0])
    
    fig, axes = plt.subplots(rows, per_colums, figsize=(3*per_colums, 4*rows))
    fig.suptitle(f"Cluster {cluster_num}", fontsize=50, color="r")

    for i, index in enumerate(selected_indices):
        data = bubble_region[index]
        
        for j, sliced_data in enumerate(data):
            if j == 0:
                axes[i, j].set_title(f"No:{index}", fontsize=30)
            
            axes[i, j].imshow(sliced_data, cmap="jet", vmin=0, vmax=1)
            axes[i, j].axis("off")
            
    # レイアウトを調整して、タイトルと画像が重ならないようにする
    plt.tight_layout(rect=[0, 0.03, 1, 0.98])
    plt.savefig(f"{match[0][1:-1]}_{log_name}_bubble_cluster{cluster_num}.png", dpi=70)
    plt.show()